In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-11-09T17:10:01.788357+00:00!"


In [2]:
import pandas as pd
import os 
import sqlalchemy
import time

In [3]:
with open("/home/kali/COVID-19-ES/airflow/process/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/kali/COVID-19-ES/airflow/download_file/downloads/Informe_Comunicacion_20211025.ods


In [4]:
#Here we load the excel file download for the official web site the health ministry
#Remove the 3 last lines Total, vaccins of Armed forces and Health professionals
#Leaving the sheet_name empty refers will take the first sheet in the excel file
dataframe = pd.read_excel(input_file)
dataframe

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Dosis entregadas Janssen (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Nº Personas con dosis adicional,Fecha de la última vacuna registrada (2)
0,Andalucía,9434970,2019500,1528900,333500,13316870,12911991,0.969597,6837216,6728587,94695,2021-10-24 00:00:00
1,Aragón,1531845,275200,265100,64700,2136845,2004019,0.937840,1065702,1046295,19334,2021-10-23 00:00:00
2,Asturias,1224725,162000,220900,43000,1650625,1677703,1.016405,873504,864580,13853,2021-10-22 00:00:00
3,Baleares,1272150,305100,256400,57600,1891250,1605357,0.848834,855619,838207,6349,2021-10-24 00:00:00
4,Canarias,2358690,574700,455800,133650,3522840,3155026,0.895592,1677222,1636507,13069,2021-10-24 00:00:00
5,Cantabria,727305,114900,125200,22200,989605,920951,0.930625,481975,475814,7627,2021-10-22 00:00:00
6,Castilla y Leon,2916525,412400,481200,102200,3912325,3793355,0.969591,1999544,1959795,62850,2021-10-20 00:00:00
7,Castilla La Mancha,2330505,446800,394900,95075,3267280,3025617,0.926035,1617085,1552172,22165,2021-10-23 00:00:00
8,Cataluña,8368460,1604900,1514300,375750,11863410,11152228,0.940052,5962893,5832442,73718,2021-10-24 00:00:00
9,C. Valenciana,5797540,1213650,863900,213450,8088540,7657132,0.946664,4070533,4003935,40404,2021-10-24 00:00:00


In [5]:
cols_names_dict = {'Unnamed: 0': 'community',
             'Total Dosis entregadas (1)': 'Total doses delivered',
             'Dosis administradas (2)': 'Doses administered',
             '% sobre entregadas': '% over delivered',
             'Nº Personas vacunadas(pauta completada)': 'No. People vaccinated',
             'Fecha de la última vacuna registrada (2)': 'Date'}

In [6]:
#rename columns
dataframe.rename(columns=cols_names_dict, inplace=True)
dataframe = dataframe[list(cols_names_dict.values())]
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,Date
0,Andalucía,13316870,12911991,0.969597,6728587,2021-10-24 00:00:00
1,Aragón,2136845,2004019,0.937840,1046295,2021-10-23 00:00:00
2,Asturias,1650625,1677703,1.016405,864580,2021-10-22 00:00:00
3,Baleares,1891250,1605357,0.848834,838207,2021-10-24 00:00:00
4,Canarias,3522840,3155026,0.895592,1636507,2021-10-24 00:00:00
5,Cantabria,989605,920951,0.930625,475814,2021-10-22 00:00:00
6,Castilla y Leon,3912325,3793355,0.969591,1959795,2021-10-20 00:00:00
7,Castilla La Mancha,3267280,3025617,0.926035,1552172,2021-10-23 00:00:00
8,Cataluña,11863410,11152228,0.940052,5832442,2021-10-24 00:00:00
9,C. Valenciana,8088540,7657132,0.946664,4003935,2021-10-24 00:00:00


In [7]:
communities = ['Andalucía',
         'Aragón',
         'Asturias ',
         'Baleares',
         'C. Valenciana',
         'Canarias',
         'Cantabria',
         'Castilla La Mancha',
         'Castilla y Leon ',
         'Cataluña',
         'Ceuta',
         'Extremadura',
         'Galicia',
         'La Rioja',
         'Madrid',
         'Melilla',
         'Murcia ',
         'Navarra',
         'País Vasco'
]
dataframe = dataframe.loc[dataframe["community"].isin(communities)]
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,Date
0,Andalucía,13316870,12911991,0.969597,6728587,2021-10-24 00:00:00
1,Aragón,2136845,2004019,0.937840,1046295,2021-10-23 00:00:00
2,Asturias,1650625,1677703,1.016405,864580,2021-10-22 00:00:00
3,Baleares,1891250,1605357,0.848834,838207,2021-10-24 00:00:00
4,Canarias,3522840,3155026,0.895592,1636507,2021-10-24 00:00:00
5,Cantabria,989605,920951,0.930625,475814,2021-10-22 00:00:00
6,Castilla y Leon,3912325,3793355,0.969591,1959795,2021-10-20 00:00:00
7,Castilla La Mancha,3267280,3025617,0.926035,1552172,2021-10-23 00:00:00
8,Cataluña,11863410,11152228,0.940052,5832442,2021-10-24 00:00:00
9,C. Valenciana,8088540,7657132,0.946664,4003935,2021-10-24 00:00:00


In [8]:
#Add ccaa_iso columns (iso code of communities)
def format_community(community_name) : 
    communities = {
        "Andalucía": "AN" ,
        "Aragón": "AR",
        "Asturias": "AS",
        "Canarias": "CN",
        "Cantabria": "CB",
        "Castilla y Leon": "CL",
        "Castilla La Mancha": "CM",
        "Cataluña": "CT",
        "Extremadura": "EX",
        "Galicia": "GA",
        "Baleares": "IB",
        "La Rioja": "RI",
        "Madrid": "MD",
        "Murcia": "MC",
        "Navarra": "NC",
        "País Vasco" : "PV",
        "C. Valenciana": "VC",
        "Ceuta": "CE", 
        "Melilla" : "ML"
    }
    return communities[community_name.strip()] if community_name.strip() in list(communities.keys()) else community_name

dataframe["ccaa_iso"] = dataframe["community"].apply(format_community)
#Set index to date
dataframe = dataframe.set_index("Date")
dataframe

/home/kali/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,ccaa_iso
Date,,,,,,
2021-10-24,Andalucía,13316870,12911991,0.969597,6728587,AN
2021-10-23,Aragón,2136845,2004019,0.937840,1046295,AR
2021-10-22,Asturias,1650625,1677703,1.016405,864580,AS
2021-10-24,Baleares,1891250,1605357,0.848834,838207,IB
2021-10-24,Canarias,3522840,3155026,0.895592,1636507,CN
2021-10-22,Cantabria,989605,920951,0.930625,475814,CB
2021-10-20,Castilla y Leon,3912325,3793355,0.969591,1959795,CL
2021-10-23,Castilla La Mancha,3267280,3025617,0.926035,1552172,CM
2021-10-24,Cataluña,11863410,11152228,0.940052,5832442,CT


In [9]:
def connection(attempts = 3):
    try :
        database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://u5487210@cvp19:jRG2XXi4CwjHvAa@cvp19.mysql.database.azure.com/my_db')
        return database_connection
    except : 
        if attempts>0:
            print("Error!")
            time.sleep(30)
            connection(attempts-1)
    return False


In [10]:
if connection() : 
    dataframe.to_sql(con=connection(), name='vaccins', if_exists='append')
else : 
    print("The error persists")

In [11]:
os.system(f'mv "{input_file}" "/home/kali/COVID-19-ES/airflow/process/processed/"')

0